In [9]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:25pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:21pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

<b><font color="red" size="6">ch11. 데이터프레임과 시리즈(Pandas)</font></b>
# 9절. 데이터 그룹화 및 집계

In [2]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
cf. pivot_table

In [16]:
# 1) 그룹화 방법1
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].기초통계함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할열을 제외한 모든 열 적용  
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
11,4.8,3.4,1.6,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
81,5.5,2.4,3.7,1.0,versicolor
98,5.1,2.5,3.0,1.1,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
134,6.1,2.6,5.6,1.4,virginica
148,6.2,3.4,5.4,2.3,virginica


In [32]:
display(iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']])
display(iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean())

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [29]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [28]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [24]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [31]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [ ]:
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법2
#     df.pivot_table(index=[그룹핑할열],
    #                values=[함수적용해서 출력할 열들],
    #                aggfunc=함수(기본값:mean))

In [37]:
iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [39]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width'],
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [41]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [42]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [3]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [4]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [5]:
# 각열의 임의의 데이터에 결측치를 할당
# iris.iloc[0, 0] = np.nan

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0


In [6]:
import random
print(random.sample(range(149), 3))
for col in range(iris.shape[1]-2): # iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan

[2, 125, 100]


In [7]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [11]:
iris.groupby(by=['Species','num'], 
            # as_index=False
            )[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              24           25
           3              24           24
virginica  4              25           25
           5              25           24

In [12]:
iris.pivot_table(index=['Species', 'num'],
                values = ['Petal.Length', 'Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              24           25
           3              24           24
virginica  4              25           25
           5              25           24

In [14]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  49.000000   50.000000  50.000000
             mean    5.000000    5.936000   6.588000
             std     0.353553    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  49.000000   49.000000  50.000000
             mean    3.418367    2.779592   2.974000
             std     0.376759    0.309556   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.600000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.600000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  49.000000   48.000000  50.000000
             mean    1.465306    4.256250   5.552000
             std     0.173867    0.465813   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.600000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
Petal.Width  count  48.000000   49.000000  49.000000
             mean    0.245833    1.326531   2.024490
             std     0.107106    0.199766   0.277286
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [18]:
iris.drop('num', axis=1, inplace=True)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa


In [21]:
# 레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [24]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                  package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [38]:
air_melted =  airquality.melt(id_vars=['Month','Day'],
               var_name='변수명', # 기본값:variable
               value_name='값') # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [40]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷) \\192.168.0.25
%ls D:\ai_x\download\shareData\부동산_250213\

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: BE7A-ADFF

 D:\ai_x\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 02:05    <DIR>          .
2025-04-30  오후 02:05    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  491,855,675,392 바이트 남음


In [108]:
import pandas as pd
df = pd.read_csv(r'D:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                encoding='cp949')
df.head(1)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,...,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,...,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443


In [109]:
new_df = df.melt(id_vars='지역',
           var_name='연도월',
           value_name='평당분양가')
new_df.head()

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [51]:
# air_melted를 와이드 포맷으로
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [57]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                      columns='변수명',
                      values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [58]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터프레임이나 시리즈에서 가능)
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능)
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [67]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [74]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [78]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [79]:
get_year('1999-05-01')

1999

In [80]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [81]:
get_month('1999-05-01')

5

In [84]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [87]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.846980
Sepal.Width     3.056757
Petal.Length    3.766667
Petal.Width     1.205479
dtype: float64

In [89]:
X.iloc[0] -avg

Sepal.Length   -0.746980
Sepal.Width     0.443243
Petal.Length   -2.366667
Petal.Width    -1.005479
dtype: float64

In [91]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.75,0.44,-2.37,-1.01
1,-0.95,-0.06,-2.37,-1.01
2,-1.15,0.14,-2.47,-1.01
3,-1.25,0.04,-2.27,-1.01
4,-0.85,0.54,-2.37,-1.01
...,...,...,...,...
145,0.85,-0.06,1.43,1.09
146,0.45,-0.56,1.23,0.69
147,0.65,-0.06,1.43,0.79
148,0.35,0.34,1.63,1.09


## 11.2 applymap

In [96]:
X = iris.iloc[:, :-2]

In [92]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3  map
- 1차원

In [100]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [170]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬럼을 추가하세요
# ★ 1차원 시리즈로 처리
new_df.sample()

,지역,연도월,평당분양가
165,전북,2014년9월,6719


In [175]:
new_df['연도'] = new_df['연도월'].apply(lambda str:int(str.split('년')[0]))
new_df['월'] = new_df['연도월'].map(lambda str:int(str.split('년')[1][:-1]))
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      357 non-null    object
 1   연도월     357 non-null    object
 2   평당분양가   357 non-null    int64 
 3   연도      357 non-null    int64 
 4   월       357 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 14.1+ KB


In [176]:
new_df.sample()

,지역,연도월,평당분양가,연도,월
313,경기,2015년6월,10573,2015,6


In [177]:
new_df.drop(['연도','월'], axis=1, inplace=True)

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [ ]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [186]:
new_df.head(2)

,지역,연도월,평당분양가
0,서울,2013년12월,18189
1,부산,2013년12월,8111


In [195]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 reutrn'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [196]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [199]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [201]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [202]:
# 모든 결측치를 0 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [203]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [204]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [205]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [207]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [209]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 90}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [ ]:
# ★ ★ ★ 총괄 QUIZ ★ ★ ★
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당한다
        # 결측치가 각 열마다 20개씩인지 확인
# (2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
# (2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)로 대체한다 : fillna사용X

In [10]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen

(150, 5)

In [11]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당후 확인
print('임의의 20개 index 추출 :',random.sample(range(rowlen), 20))

임의의 20개 index 추출 : [6, 41, 105, 18, 112, 57, 15, 89, 118, 117, 95, 8, 20, 75, 124, 116, 0, 69, 107, 2]


In [12]:
# 결측치 할당
for col in range(collen-1):
    iris.iloc[random.sample(range(rowlen), 20), col] = np.nan

In [20]:
# 결측치 확인 - iris.isna(), iris.isnull():결측치인지 여부
# iris.isna().sum()
print(iris.isnull().sum())
iris.tail()

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64


,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,NaN,virginica
147,6.5,3.0,NaN,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,NaN,1.8,virginica


In [31]:
# 결측치가 하나라도 있는 행 출력
iris[iris['sepal_length'].isna() | iris['sepal_width'].isna() | 
     iris['petal_length'].isna() | iris['petal_width'].isna()]

,sepal_length,sepal_width,petal_length,petal_width,species
2,NaN,NaN,1.3,0.2,setosa
11,4.8,3.4,NaN,0.2,setosa
13,4.3,NaN,1.1,0.1,setosa
14,5.8,4.0,1.2,NaN,setosa
15,5.7,4.4,1.5,NaN,setosa
...,...,...,...,...,...
142,NaN,2.7,5.1,1.9,virginica
144,6.7,NaN,5.7,2.5,virginica
146,6.3,2.5,5.0,NaN,virginica
147,6.5,3.0,NaN,2.0,virginica


In [25]:
# (2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
#iris.median(numeric_only=True)
#iris.iloc[:, :-1].median()
iris.fillna(iris.iloc[:, :-1].median())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,5.8,3.0,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.3,virginica
147,6.5,3.0,4.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
# iris 열별 중위수

In [ ]:
# 결측치 열별 중위수로 대체

In [ ]:
# (2-2단계) 결측치는 종별 열별 중위수(groupby, pivot_table)로 대체한다 : fillna사용X

In [ ]:
# 종별 열별 중위수

In [ ]:
# fillna사용X 확인

In [ ]:
# iris 한행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수

In [ ]:
# apply 적용

## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용